In [ ]:
# Setting options for the plots
%matplotlib inline
%config InlineBackend.figure_formats={'retina', 'svg'}
%config InlineBackend.rc={'savefig.dpi': 150}

In [ ]:
## filter out warnings from third-party libraries to prevent them
# from showing up in the notebooks in multiple places
import warnings

# warning from shap about tqdm 
from tqdm import TqdmExperimentalWarning
warnings.filterwarnings("ignore", 
                        category=TqdmExperimentalWarning, 
                        message="Using `tqdm.autonotebook.tqdm` .*", 
                        module="shap.explainers._linear")



In [ ]:
import itertools
import math
import os
import re
import pickle
import platform
import time

from functools import partial
from os.path import abspath, relpath, exists, join

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
from matplotlib import pyplot as plt
from textwrap import wrap

# allow older versions of pandas to work
try:
    from pandas.io.common import DtypeWarning
except ImportError:
    from pandas.errors import DtypeWarning

from IPython import sys_info
from IPython.display import display, HTML, Image, Javascript, Markdown, SVG
from rsmtool.reader import DataReader
from rsmtool.writer import DataWriter
from rsmtool.utils.files import parse_json_with_comments
from rsmtool.utils.notebook import (float_format_func,
                                    int_or_float_format_func,
                                    compute_subgroup_plot_params,
                                    bold_highlighter,
                                    color_highlighter,
                                    show_thumbnail)

from rsmtool.fairness_utils import (get_fairness_analyses,
                                    write_fairness_results)

from rsmtool.version import VERSION as rsmtool_version

# turn off interactive plotting
plt.ioff()

sns.set_context('notebook')

In [ ]:
rsm_report_dir = os.environ.get('RSM_REPORT_DIR', None)
if rsm_report_dir is None:
    rsm_report_dir = os.getcwd()

rsm_environ_config = join(rsm_report_dir, '.environ.json')
if not exists(rsm_environ_config):
    raise FileNotFoundError('The file {} cannot be located. '
                            'Please make sure that either (1) '
                            'you have set the correct directory with the `RSM_REPORT_DIR` '
                            'environment variable, or (2) that your `.environ.json` '
                            'file is in the same directory as your notebook.'.format(rsm_environ_config))
    
environ_config = parse_json_with_comments(rsm_environ_config)

<style type="text/css">
  div.prompt.output_prompt { 
    color: white; 
  }
  
  span.highlight_color {
    color: red;
  }
  
  span.highlight_bold {
    font-weight: bold;  
  }
    
  @media print {
    @page {
      size: landscape;
      margin: 0cm 0cm 0cm 0cm;
    }

    * {
      margin: 0px;
      padding: 0px;
    }

    #toc {
      display: none;
    }

    span.highlight_color, span.highlight_bold {
        font-weight: bolder;
        text-decoration: underline;
    }

    div.prompt.output_prompt {
      display: none;
    }
    
    h3#Python-packages, div#packages {
      display: none;
  }
</style>

In [ ]:
# NOTE: you will need to set the following manually
# if you are using this notebook interactively.
experiment_id = environ_config.get('EXPERIMENT_ID')
description = environ_config.get('DESCRIPTION')
context = environ_config.get('CONTEXT')
train_file_location = environ_config.get('TRAIN_FILE_LOCATION')
test_file_location = environ_config.get('TEST_FILE_LOCATION')
output_dir = environ_config.get('OUTPUT_DIR')
figure_dir = environ_config.get('FIGURE_DIR')
model_name = environ_config.get('MODEL_NAME')
model_type = environ_config.get('MODEL_TYPE')
skll_fixed_parameters = environ_config.get('SKLL_FIXED_PARAMETERS')
skll_objective = environ_config.get('SKLL_OBJECTIVE')
file_format = environ_config.get('FILE_FORMAT')
length_column = environ_config.get('LENGTH_COLUMN')
second_human_score_column = environ_config.get('H2_COLUMN')
use_scaled_predictions = environ_config.get('SCALED')
min_score = environ_config.get("MIN_SCORE")
max_score = environ_config.get("MAX_SCORE")
standardize_features = environ_config.get('STANDARDIZE_FEATURES')
exclude_zero_scores = environ_config.get('EXCLUDE_ZEROS')
feature_subset_file = environ_config.get('FEATURE_SUBSET_FILE', ' ')
min_items = environ_config.get('MIN_ITEMS')
use_thumbnails = environ_config.get('USE_THUMBNAILS')
predict_expected_scores = environ_config.get('PREDICT_EXPECTED_SCORES')
rater_error_variance = environ_config.get("RATER_ERROR_VARIANCE")

# groups for analysis by prompt or subgroup.
groups_desc = environ_config.get('GROUPS_FOR_DESCRIPTIVES') 
groups_eval = environ_config.get('GROUPS_FOR_EVALUATIONS') 

# min number of n for group to be displayed in the report
min_n_per_group = environ_config.get('MIN_N_PER_GROUP')

if min_n_per_group is None:
    min_n_per_group = {}

# javascript path
javascript_path = environ_config.get("JAVASCRIPT_PATH")

# Experiment Report 

In [ ]:
# initialize counter for thumbnail IDs
id_generator = itertools.count(1)

In [ ]:
with open(join(javascript_path, "sort.js"), "r", encoding="utf-8") as sortf:
    display(Javascript(data=sortf.read()))

In [ ]:
Markdown('''This report presents the analysis for **{}**: {}'''.format(experiment_id, description))

In [ ]:
markdown_str = ''
if use_thumbnails:
    markdown_str += ("""\n  - Images in this report have been converted to """
                     """clickable thumbnails.""")
if predict_expected_scores:
    markdown_str += ("""\n  - Predictions analyzed in this report are *expected scores*, """
                     """i.e., probability-weighted averages over all score points.""")

if markdown_str:
    markdown_str = '**Notes**:' + markdown_str
    display(Markdown(markdown_str))

In [ ]:
HTML(time.strftime('%c'))

In [ ]:
%%html
<div id="toc"></div>

In [ ]:
# Read in the training and testing features, both raw and pre-processed
# Make sure that the `spkitemid` and `candidate` columns are read as strings 
# to preserve any leading zeros
# We filter DtypeWarnings that pop up mostly in very large files

string_columns = ['spkitemid', 'candidate']
converter_dict = {column: str for column in string_columns}

with warnings.catch_warnings():
    warnings.filterwarnings('ignore', category=DtypeWarning)
    if exists(train_file_location):
        df_train_orig = DataReader.read_from_file(train_file_location)

    train_file = join(output_dir, '{}_train_features.{}'.format(experiment_id,
                                                                file_format))
    if exists(train_file):
        df_train = DataReader.read_from_file(train_file, converters=converter_dict)

    train_metadata_file = join(output_dir, '{}_train_metadata.{}'.format(experiment_id,
                                                                         file_format))    
    if exists(train_metadata_file):
        df_train_metadata = DataReader.read_from_file(train_metadata_file, converters=converter_dict)

    train_other_columns_file = join(output_dir, '{}_train_other_columns.{}'.format(experiment_id,
                                                                                   file_format))
    if exists(train_other_columns_file):
        df_train_other_columns = DataReader.read_from_file(train_other_columns_file, converters=converter_dict)

    train_length_file = join(output_dir, '{}_train_response_lengths.{}'.format(experiment_id,
                                                                               file_format))
    if exists(train_length_file):
        df_train_length = DataReader.read_from_file(train_length_file, converters=converter_dict)

    train_excluded_file = join(output_dir, '{}_train_excluded_responses.{}'.format(experiment_id,
                                                                                   file_format))
    if exists(train_excluded_file):
        df_train_excluded = DataReader.read_from_file(train_excluded_file, converters=converter_dict)

    train_responses_with_excluded_flags_file = join(output_dir, '{}_train_responses_with_excluded_flags.{}'.format(experiment_id,
                                                                                                                   file_format))
    if exists(train_responses_with_excluded_flags_file):
        df_train_responses_with_excluded_flags = DataReader.read_from_file(train_responses_with_excluded_flags_file,
                                                                           converters=converter_dict)

    train_preproc_file = join(output_dir, '{}_train_preprocessed_features.{}'.format(experiment_id,
                                                                                     file_format))    
    if exists(train_preproc_file):
        df_train_preproc = DataReader.read_from_file(train_preproc_file, converters=converter_dict)

    if exists(test_file_location):
        df_test_orig = DataReader.read_from_file(test_file_location)

    test_file = join(output_dir, '{}_test_features.{}'.format(experiment_id,
                                                              file_format))
    if exists(test_file):
        df_test = DataReader.read_from_file(test_file, converters=converter_dict)

    test_metadata_file = join(output_dir, '{}_test_metadata.{}'.format(experiment_id,
                                                                       file_format))    
    if exists(test_metadata_file):
        df_test_metadata = DataReader.read_from_file(test_metadata_file, converters=converter_dict)

    test_other_columns_file = join(output_dir, '{}_test_other_columns.{}'.format(experiment_id,
                                                                                 file_format))
    if exists(test_other_columns_file):
        df_test_other_columns = DataReader.read_from_file(test_other_columns_file, converters=converter_dict)

    test_human_scores_file = join(output_dir, '{}_test_human_scores.{}'.format(experiment_id,
                                                                               file_format))
    if exists(test_human_scores_file):
        df_test_human_scores = DataReader.read_from_file(test_human_scores_file, converters=converter_dict)

    test_excluded_file = join(output_dir, '{}_test_excluded_responses.{}'.format(experiment_id,
                                                                                 file_format))
    if exists(test_excluded_file):
        df_test_excluded = DataReader.read_from_file(test_excluded_file, converters=converter_dict)

    test_responses_with_excluded_flags_file = join(output_dir, '{}_test_responses_with_excluded_flags.{}'.format(experiment_id,
                                                                                                                 file_format))
    if exists(test_responses_with_excluded_flags_file):
        df_test_responses_with_excluded_flags = DataReader.read_from_file(test_responses_with_excluded_flags_file,
                                                                          converters=converter_dict)

    test_preproc_file = join(output_dir, '{}_test_preprocessed_features.{}'.format(experiment_id,
                                                                                   file_format))
    if exists(test_preproc_file):
        df_test_preproc = DataReader.read_from_file(test_preproc_file, converters=converter_dict)

    pred_preproc_file = join(output_dir, '{}_pred_processed.{}'.format(experiment_id,
                                                                       file_format))
    if exists(pred_preproc_file):
        df_pred_preproc = DataReader.read_from_file(pred_preproc_file, converters=converter_dict)

    feature_file = join(output_dir, '{}_feature.{}'.format(experiment_id,
                                                           file_format))
    if exists(feature_file):
        df_features = DataReader.read_from_file(feature_file, converters=converter_dict)
        features_used = [c for c in df_features.feature.values]
        # compute the longest feature name: we'll need if for the plots
        longest_feature_name = max(map(len, features_used))

    betas_file = join(output_dir, '{}_betas.{}'.format(experiment_id,
                                                       file_format))
    if exists(betas_file):
        df_betas = DataReader.read_from_file(betas_file)

    if exists(feature_subset_file):
        df_feature_subset_specs = DataReader.read_from_file(feature_subset_file)
    else:
        df_feature_subset_specs = None

In [ ]:
# check for continuous human scores in the evaluation set
continuous_human_score = False

if exists(pred_preproc_file):
    if not df_pred_preproc['sc1'].equals(np.round(df_pred_preproc['sc1'])):
        continuous_human_score = True

## Description of the data

In [ ]:
try:
    num_excluded_train = len(df_train_responses_with_excluded_flags)
except NameError:
    num_excluded_train = 0

try:
    num_excluded_test = len(df_test_responses_with_excluded_flags)
except NameError:
    num_excluded_test = 0

if context == 'rsmtool':
    pct_excluded_train = round(100*num_excluded_train/len(df_train_orig), 2)
pct_excluded_test = round(100*num_excluded_test/len(df_test_orig), 2)

if (num_excluded_train != 0 or num_excluded_test != 0):
    display(Markdown("### Responses excluded due to flags"))

    display(Markdown("Total number of responses excluded due to flags:"))
    if context=='rsmtool':
        display(Markdown("Training set: {} responses ({:.1f}% of the original {} responses)".format(num_excluded_train, pct_excluded_train, len(df_train_orig))))
    display(Markdown("Evaluation set: {} responses ({:.1f}% of the original {} responses)".format(num_excluded_test, pct_excluded_test, len(df_test_orig))))


### Responses excluded due to non-numeric feature values or scores

In [ ]:
try:
    num_missing_rows_train = len(df_train_excluded)
except NameError:
    num_missing_rows_train = 0

if context == 'rsmtool':
    pct_missing_rows_train = 100*num_missing_rows_train/len(df_train_orig)

try:
    num_missing_rows_test = len(df_test_excluded)
except:
    num_missing_rows_test = 0
pct_missing_rows_test = 100*num_missing_rows_test/len(df_test_orig)

In [ ]:
excluded_candidates_note = Markdown("Note: if a candidate had less than {} responses left for analysis after applying all filters, "
                                    "all responses from that "
                                    "candidate were excluded from further analysis.".format(min_items))

In [ ]:
if context == 'rsmtool':
    display(Markdown("**Training set**"))
    display(Markdown('Total number of excluded responses: {} ({:.1f}% of the original {})'.format(num_missing_rows_train, pct_missing_rows_train, len(df_train_orig))))
    if num_missing_rows_train != 0:
        train_excluded_analysis_file = join(output_dir, '{}_train_excluded_composition.{}'.format(experiment_id,
                                                                                                  file_format))
        df_train_excluded_analysis = DataReader.read_from_file(train_excluded_analysis_file)
        display(HTML(df_train_excluded_analysis.to_html(classes=['sortable'], na_rep='-', float_format=float_format_func, index=False))) 
        if min_items > 0:
            display(excluded_candidates_note)

In [ ]:
display(Markdown('**Evaluation set**'))
display(Markdown('Total number of excluded responses: {} ({:.1f}% of the original {})'.format(num_missing_rows_test, pct_missing_rows_test, len(df_test_orig))))
if num_missing_rows_test != 0:
    test_excluded_analysis_file = join(output_dir, '{}_test_excluded_composition.{}'.format(experiment_id,
                                                                                            file_format))
    df_test_excluded_analysis = DataReader.read_from_file(test_excluded_analysis_file)
    display(HTML(df_test_excluded_analysis.to_html(classes=['sortable'], na_rep='-', float_format=float_format_func, index=False)))
    if min_items > 0:
        display(excluded_candidates_note)

The rest of this report is based only on the responses that were not excluded above.

In [ ]:
if context == 'rsmtool':
    display(Markdown('### Composition of the training and evaluation sets'))
elif context == 'rsmeval':
    display(Markdown('### Composition of the evaluation set'))

In [ ]:
# show the table showing candidate (speaker), prompt 
# and responses stats for training and test

# feature descriptives extra table
data_composition_file = join(output_dir, '{}_data_composition.{}'.format(experiment_id, file_format))
df_data_desc = DataReader.read_from_file(data_composition_file)
display(HTML(df_data_desc.to_html(classes=['sortable'], float_format=float_format_func, index=False)))

try:
    num_double_scored_responses = len(df_test_human_scores[df_test_human_scores['sc2'].notnull()])
except NameError:
    pass
else:
    zeros_included_or_excluded = 'excluded' if exclude_zero_scores else 'included'
    display(Markdown("Total number of double scored responses in the evaluation set" 
                     " used: {} (zeros {})".format(num_double_scored_responses,
                                                   zeros_included_or_excluded)))

## Overall descriptive feature statistics

These values are reported before transformations.

In [ ]:
# feature descriptives table
desc_file = join(output_dir, '{}_feature_descriptives.{}'.format(experiment_id, file_format))

df_desc = DataReader.read_from_file(desc_file, index_col=0)
HTML(df_desc.to_html(classes=['sortable'], float_format=float_format_func))

### Prevalence of recoded cases

This sections shows the number and percentage of cases truncated to mean +/- 4 SD for each feature.

In [ ]:
outliers_file = join(output_dir, '{}_feature_outliers.{}'.format(experiment_id, file_format))
df_outliers = DataReader.read_from_file(outliers_file, index_col=0)
df_outliers.index.name = 'feature'
df_outliers = df_outliers.reset_index()
df_outliers = pd.melt(df_outliers, id_vars=['feature'])
df_outliers = df_outliers[df_outliers.variable.str.contains(r'[ulb].*?perc')]


# we need to increase the plot height if feature names are long
if longest_feature_name > 10:
    height = 3 + math.ceil((longest_feature_name - 10)/10)
else:
    height = 3
    
# we also need a higher aspect if we have more than 40 features
# The aspect defines the final width of the plot (width=aspect*height).
# We keep the width constant (9 for plots with many features or 6
# for plots with few features) by dividing the expected width
# by the height. 
aspect = 9/height if len(features_used) > 40 else 6/height


# colors for the plot
colors = sns.color_palette("Greys", 3)

# what's the largest value in the data frame
maxperc = df_outliers['value'].max()

# compute the limits for the graph
limits = (0, max(2.5, maxperc))

with sns.axes_style('whitegrid'):
    # create a barplot without a legend since we will manually
    # add one later
    p = sns.catplot(x="feature", y="value", hue="variable", kind="bar", 
                    palette=colors, data=df_outliers, height=height, 
                    aspect=aspect, legend=False)
    p.set_axis_labels('', '% cases truncated\nto mean +/- 4*sd')
    p.set_xticklabels(rotation=90)
    p.set(ylim=limits)

    # add a line at 2%
    axis = p.axes[0][0]
    axis.axhline(y=2.0, linestyle='--', linewidth=1.5, color='black')

    # add a legend with the right colors
    legend=axis.legend(('both', 'lower', 'upper'), title='', frameon=True, fancybox=True, ncol=3)
    legend.legend_handles[0].set_color(colors[0])
    legend.legend_handles[1].set_color(colors[1])

    # we want to try to force `tight_layout()`, but if this 
    # raises a warning, we don't want the entire notebook to fail
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        plt.tight_layout(h_pad=1.0)

    imgfile = join(figure_dir, '{}_outliers.svg'.format(experiment_id))
    plt.savefig(imgfile)
    if use_thumbnails:
        show_thumbnail(imgfile, next(id_generator))
    else:
        plt.show()

### Feature value distribution

The following table shows additional statistics for the data. Quantiles are computed using type=3 method used in SAS. The mild outliers are defined as data points between [1.5, 3) \* IQR away from the nearest quartile. Extreme outliers are the data points >= 3 * IQR away from the nearest quartile.

In [ ]:
# feature descriptives extra table
desce_file = join(output_dir, '{}_feature_descriptivesExtra.{}'.format(experiment_id,
                                                                       file_format))
df_desce = DataReader.read_from_file(desce_file, index_col=0)
HTML(df_desce.to_html(classes=['sortable'], float_format=float_format_func))

##  Feature distributions and inter-feature correlations

### Training set distributions

The following plot shows the distributions of the feature values in 
the training set, after transformation (if applicable), truncation 
and standardization (if applicable). The line shows the kernel density estimate. The 
human score (`sc1`) is also included. 

Response length (`length`) is included if you specified `length_column` in the config file, unless
the column had missing values or a standard deviation <= 0.

In [ ]:
selected_columns = features_used + ['sc1', 'spkitemid']
df_train_preproc_selected_features = df_train_preproc[selected_columns]
try:
    df_train_preproc_selected_features = df_train_preproc_selected_features.merge(df_train_length, on='spkitemid')
except NameError:
    column_order = sorted(features_used) + ['sc1']
else:
    column_order = sorted(features_used) + ['sc1', 'length']

df_train_preproc_melted = pd.melt(df_train_preproc_selected_features, id_vars=['spkitemid'])
df_train_preproc_melted = df_train_preproc_melted[['variable', 'value']]

# we need to reduce col_wrap and increase width if the feature names are too long
if longest_feature_name > 10:
    col_wrap = 2
    # adjust height to allow for wrapping really long names. We allow 0.25 in per line
    height = 2+(math.ceil(longest_feature_name/30)*0.25)
    aspect = 4/height
else:
    col_wrap = 3
    aspect = 1


with warnings.catch_warnings():
    warnings.simplefilter('ignore')

    with sns.axes_style('white'):
        g = sns.FacetGrid(col='variable', data=df_train_preproc_melted, col_wrap=col_wrap, 
                          col_order=column_order, sharex=False, sharey=False, height=height, 
                          aspect=aspect)
        g.map(sns.distplot, "value", color="grey", kde=False)
        for ax, cname in zip(g.axes, g.col_names):
            labels = ax.get_xticks()
            ax.set_xlabel('')
            ax.set_xticks(ax.get_xticks(), labels, rotation=90)
            plot_title = '\n'.join(wrap(str(cname), 30))
            ax.set_title(plot_title)

        plt.tight_layout(h_pad=1.0)
        imgfile = join(figure_dir, '{}_distrib.svg'.format(experiment_id))
        plt.savefig(imgfile)
        if use_thumbnails:
            show_thumbnail(imgfile, next(id_generator))
        else:
            plt.show()

### Inter-feature correlations

The following table shows the Pearson correlations between all the training features
after transformation (if applicable), truncation and standardization (if applicable). The human score 
(`sc1`) is also included. 

Response length (`length`) is included if 
you specified `length_column` in the config file, unless the column had missing 
values or a standard deviation <= 0. 

The following values are <span class="highlight_color">highlighted</span>:
- inter-feature correlations above 0.7, and
- `sc1`-feature correlations lower than 0.1 or higher than 0.7

In [ ]:
cors_file = join(output_dir, '{}_cors_processed.{}'.format(experiment_id,
                                                           file_format))
df_cors = DataReader.read_from_file(cors_file, index_col=0)
if 'length' in df_cors.columns:
    feature_columns = sorted([c for c in df_cors.columns if c not in ['sc1', 'length']])
    order = ['sc1', 'length'] + feature_columns
else:
    feature_columns = sorted([c for c in df_cors.columns if c != 'sc1'])
    order = ['sc1'] + feature_columns
    
df_cors = df_cors.reindex(index=order, columns=order)

# wrap the column names if the feature names are very long
if longest_feature_name > 10:
    column_names = ['\n'.join(wrap(c, 10)) for c in order]
else:
    column_names = order
    
df_cors.columns = column_names

# apply two different formatting to the columns according
# to two different thresholds. The first one highlights all
# inter-feature correlations > 0.7 (so, not including sc1)
# and the second highlights all sc1-X correlations lower
# than 0.1 and higher than 0.7. We will use red for the
# first formatting and blue for the second one. 
formatter1 = partial(color_highlighter, low=-1, high=0.7)
formatter2 = partial(color_highlighter, low=0.1, high=0.7)

formatter_dict = {c: formatter1 for c in column_names if not c == 'sc1'}
formatter_dict.update({'sc1': formatter2})

HTML(df_cors.to_html(classes=['sortable'], formatters=formatter_dict, escape=False))

### Marginal and partial correlations

The plot below shows correlations between truncated and standardized (if applicable) values of each feature against human score. The first bar (`Marginal`) in each case shows Pearson's correlation. The second bar (`Partial - all`) shows partial correlations after controlling for all other variables. If you specified `length_column` in the config file, a third bar (`Partial - length`) will show partial correlations of each feature against the human score after controlling for length. The dotted lines correspond to *r* = 0.1 and *r* = 0.7.

In [ ]:
# read in and merge the score correlations 
margcor_file = join(output_dir, '{}_margcor_score_all_data.{}'.format(experiment_id, file_format))
pcor_file = join(output_dir, '{}_pcor_score_all_data.{}'.format(experiment_id, file_format))

df_margcor = DataReader.read_from_file(margcor_file, index_col=0)
df_pcor = DataReader.read_from_file(pcor_file, index_col=0)

# check if we have length partial correlations
pcor_no_length_file = join(output_dir, '{}_pcor_score_no_length_all_data.{}'.format(experiment_id,
                                                                                    file_format))
with_length = exists(pcor_no_length_file)
if with_length:
    df_pcor_no_length = DataReader.read_from_file(pcor_no_length_file, index_col=0)
    df_mpcor = pd.DataFrame([df_margcor.loc['All data'], 
                             df_pcor.loc['All data'], 
                             df_pcor_no_length.loc['All data']]).transpose()
    df_mpcor.columns = ['marginal', 'partial_all', 'partial_length']
    num_entries = 3
    labels = ('Marginal', 'Partial - all', 'Partial - length')

else:
    df_mpcor = pd.DataFrame([df_margcor.loc['All data'], 
                             df_pcor.loc['All data']]).transpose()
    df_mpcor.columns = ['marginal', 'partial_all']
    num_entries = 2
    labels = ('Marginal', 'Partial (all)')

df_mpcor.index.name = 'feature'
df_mpcor = df_mpcor.reset_index()
df_mpcor = pd.melt(df_mpcor, id_vars=['feature'])

# we need to change the plot height if the feature names are long
if longest_feature_name > 10:
    height = 3 + math.ceil((longest_feature_name - 10)/10)
else:
    height = 3
        
# we need a higher aspect if we have more than 40 features
aspect = 9/height if len(features_used) > 40 else 6/height


# get the colors for the plot
colors = sns.color_palette("Greys", num_entries)

# check for any negative correlations
limits = (0, 1)
if len(df_mpcor[df_mpcor.value < 0]):
    limits = (-1, 1)


with sns.axes_style('whitegrid'):

    # generate a bar plot but without the legend since we will
    # manually add one later
    p = sns.catplot(x="feature", y="value", hue="variable", kind="bar",
                    palette=colors, data=df_mpcor, height=height, 
                    aspect=aspect, legend=False)
    p.set_axis_labels('', 'Correlation with score')
    p.set_xticklabels(rotation=90)
    p.set(ylim=limits)
    
    # add a line at 0.1 and 0.7
    axis = p.axes[0][0]
    axis.axhline(y=0.1, linestyle='--', linewidth=0.5, color='black');
    axis.axhline(y=0.7, linestyle='--', linewidth=0.5, color='black');

    # create the legend manually with the right colors
    legend = axis.legend(labels=labels, title='', frameon=True, 
                         fancybox=True, ncol=num_entries)
    for i in range(num_entries):
        legend.legend_handles[i].set_color(colors[i]);

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        plt.tight_layout(h_pad=1.0)

    imgfile = join(figure_dir, '{}_cors_score.svg'.format(experiment_id))
    plt.savefig(imgfile)
    if use_thumbnails:
        show_thumbnail(imgfile, next(id_generator))
    else:
        plt.show()

In [ ]:
len_margcor_file = join(output_dir, '{}_margcor_length_all_data.{}'.format(experiment_id,
                                                                           file_format))
len_pcor_file = join(output_dir, '{}_pcor_length_all_data.{}'.format(experiment_id,
                                                                     file_format))
if exists(len_margcor_file) and exists(len_pcor_file):

    if standardize_features:
        display(Markdown("The plot below shows the same correlations between truncated and "
                         "standardized values of each feature against length."))
    else:
        display(Markdown("The plot below shows the same correlations between truncated and "
                         "un-standardized values of each feature against length."))

    df_margcor = DataReader.read_from_file(len_margcor_file, index_col=0)
    df_pcor = DataReader.read_from_file(len_pcor_file, index_col=0)
    df_mpcor = pd.DataFrame([df_margcor.loc['All data'], df_pcor.loc['All data']]).transpose()
    df_mpcor.index.name = 'feature'
    df_mpcor.columns = ['marginal', 'partial']
    df_mpcor = df_mpcor.reset_index()
    df_mpcor = pd.melt(df_mpcor, id_vars=['feature'])

    # we need to change the plot height if the feature names are long
    if longest_feature_name > 10:
        height = 3 + math.ceil((longest_feature_name - 10)/10)
    else:
        height = 3
        
    # we need a higher aspect if we have more than 40 features
    aspect = 9/height if len(features_used) > 40 else 6/height


    # check for any negative correlations
    limits = (0, 1)
    if len(df_mpcor[df_mpcor.value < 0]):
        limits = (-1, 1)

    # get the colors for the plot
    colors = sns.color_palette("Greys", 2)
        
    with sns.axes_style('whitegrid'):
        
        # create a barplot but without the legend since
        # we will manually add one later
        p = sns.catplot(x="feature", y="value", hue="variable", kind="bar",
                        palette=colors, data=df_mpcor, height=height, 
                        aspect=aspect, legend=False)
        p.set_axis_labels('', 'Correlation with length')
        p.set_xticklabels(rotation=90)
        p.set(ylim=limits)

        # create the legend manually with the right colors
        axis = p.axes[0][0]
        legend = axis.legend(labels=('Marginal', 'Partial  - all'), title='', 
                             frameon=True, fancybox=True, ncol=2)
        legend.legend_handles[0].set_color(colors[0]);
        legend.legend_handles[1].set_color(colors[1]);
        imgfile = join(figure_dir, '{}_cors_length.svg'.format(experiment_id))
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            plt.tight_layout(h_pad=1.0)

        plt.savefig(imgfile) 
        if use_thumbnails:
            show_thumbnail(imgfile, next(id_generator))
        else:
            plt.show()

In [ ]:
markdown_strs = ['## Consistency']

consistency_file = join(output_dir, '{}_consistency.{}'.format(experiment_id, file_format))
degradation_file = join(output_dir, '{}_degradation.{}'.format(experiment_id, file_format))
disattenuation_file = join(output_dir, '{}_disattenuated_correlations.{}'.format(experiment_id, file_format))
eval_file = join(output_dir, '{}_eval.{}'.format(experiment_id,
                                                 file_format))
confmat_h1h2_file = join(output_dir, '{}_confMatrix_h1h2.{}'.format(experiment_id, file_format))

if exists(consistency_file) and exists(degradation_file) and exists(disattenuation_file):
    df_consistency = DataReader.read_from_file(consistency_file, index_col=0)
    df_degradation = DataReader.read_from_file(degradation_file, index_col=0)
    df_dis_corrs = DataReader.read_from_file(disattenuation_file, index_col=0)
    df_eval = DataReader.read_from_file(eval_file, index_col=0)

    markdown_strs.append('*Note: this section assumes that the score used for evaluating machine scores '
                         'is the score assigned by the first rater.*')
    markdown_strs.append('### Human-human agreement')
    markdown_strs.append("This table shows the human-human agreement on the "
                         "double-scored evaluation data.")
    if continuous_human_score:
        markdown_strs.append('For the computation of `kappa` and `wtkappa` '
                             'human scores have beeen rounded to the nearest integer.')
        
    markdown_strs.append("The following are <span class='highlight_color'>highlighted </span>: ")
    markdown_strs.append(' - Exact agreement (`exact_agr`) < 50%')
    markdown_strs.append(' - Adjacent agreement (`adj_agr`) < 95%')
    markdown_strs.append(' - Quadratic weighted kappa (`wtkappa`) < 0.7')
    markdown_strs.append(' - Pearson correlation (`corr`) < 0.7')
    display(Markdown('\n'.join(markdown_strs)))
    
    # display the HTML for the table with the various formatters
    formatter_exact_agr = partial(color_highlighter, low=50, high=100)
    formatter_adj_agr = partial(color_highlighter, low=95, high=100)
    formatter_wtkappa_corr = partial(color_highlighter, low=0.7)
    formatter_dict = {'exact_agr': formatter_exact_agr, 
                      'adj_agr': formatter_adj_agr,
                      'wtkappa': formatter_wtkappa_corr, 
                      'corr': formatter_wtkappa_corr}
    display(HTML(df_consistency.to_html(index=False,
                                        escape=False,
                                        float_format=float_format_func,
                                        formatters=formatter_dict)))
    
    markdown_strs = ['### Degradation']
    markdown_strs.append('The next table shows the degradation in the evaluation metrics '
                         '(`diff`) when comparing the machine (`H-M`) to a second human (`H-H`). '
                         'A positive degradation value indicates better human-machine performance. '
                         'Note that the human-machine agreement is computed on the full '
                         'dataset (to get a reliable estimate) whereas the human-human '
                         'agreement is computed on the subset of responses that were double-scored.')
    markdown_strs.append("\nThe following degradation values are "
                         "<span class='highlight_color'>highlighted</span>")
    markdown_strs.append(' - `corr` < -0.1')
    markdown_strs.append(' - `wtkappa` < -0.1')
    display(Markdown('\n'.join(markdown_strs)))
    df_eval_for_degradation = df_eval[df_degradation.columns].copy()
    df_consistency_for_degradation = pd.concat([df_consistency]*len(df_eval), sort=True)
    df_consistency_for_degradation = df_consistency_for_degradation[df_degradation.columns].copy()
    df_consistency_for_degradation.index = df_eval_for_degradation.index

    df_consistency_for_degradation['type'] = 'H-H'
    df_eval_for_degradation['type'] = 'H-M'
    df_degradation['type'] = 'diff'

    df = pd.concat([df_consistency_for_degradation,
                    df_eval_for_degradation,
                    df_degradation], sort=True)
    df = df[['type','corr', 'kappa', 'wtkappa', 'exact_agr', 'adj_agr', 'SMD']]
    df = df.reset_index()
    df = df.set_index(['index', 'type']).sort_index(level='index')
    df.index.names = [None, None]
    
    # display the HTML for the table with the various formatters
    formatter_corr = partial(color_highlighter, low=-0.1, high=100)
    formatter_wtkappa = partial(color_highlighter, low=-0.1, high=100)
    formatter_dict = {'corr': formatter_corr, 'wtkappa': formatter_wtkappa}
    display(HTML(df.to_html(float_format=float_format_func, 
                            formatters=formatter_dict, escape=False)))

    if exists(confmat_h1h2_file):
        markdown_strs = ['### Human-human confusion matrix']
        markdown_strs.append("Confusion matrix using the two human scores (rows=human #1, columns=human #2)."
                             " Note that the matrix is computed on the subset of responses that were "
                             "double-scored.")
        if continuous_human_score:
            markdown_strs.append("Note: Human scores have beeen rounded to the nearest integer.")
        display(Markdown('\n'.join(markdown_strs)))
        df_confmat_h1h2 = DataReader.read_from_file(confmat_h1h2_file, index_col=0)
        display(HTML(df_confmat_h1h2.to_html(float_format=float_format_func, 
                                             formatters=formatter_dict, escape=False)))

    markdown_strs = ['### Disattenuated correlations']
    markdown_strs.append('The next table shows the correlations between human and machine scores, '
                         'the correlations between two human scores, '  
                         'and disattenuated correlations between human and machine scores computed as '
                         'human-machine correlations divided by the square root of human-human '
                         'correlation. '
                         'Note that the human-machine correlation is computed on the full '
                         'dataset (to get a reliable estimate) whereas the human-human '
                         'correlation is computed on the subset of responses that were double-scored.')
    markdown_strs.append("\nThe following values are "
                         "<span class='highlight_color'>highlighted</span>")
    markdown_strs.append(' - `disattenuated_corr` < 0.9')
    display(Markdown('\n'.join(markdown_strs)))
    # display the HTML for the table with the various formatters
    formatter_dis_corr = partial(color_highlighter, low=0.9)
    formatter_dict = {'corr_disattenuated': formatter_dis_corr}
    display(HTML(df_dis_corrs.to_html(index=True,
                                      escape=False,
                                      classes=['sortable'],
                                      float_format=float_format_func,
                                      formatters=formatter_dict)))
else:  
    markdown_strs.append("The configuration file did not specify "
                         "`second_human_score_column` which is necessary to compute "
                         "consistency metrics.")
    display(Markdown('\n'.join(markdown_strs)))
    
    

## Model

In [ ]:
objective_used = skll_objective if skll_objective else 'neg_mean_squared_error'
intro_markdown_str = f"""### Characteristics 

- Model: **{model_name}**
- Number of features: **{len(features_used)}**
- Tuning objective: **{objective_used}**
"""

if skll_fixed_parameters:
    intro_markdown_str += "- Custom fixed parameters:"
    for parameter, value in skll_fixed_parameters.items():
        intro_markdown_str += f"\n    - `{parameter}` : **{value!r}**"

display(Markdown(intro_markdown_str))

In [ ]:
skll_linear_models = ['Ridge', 'BayesianRidge', 'LinearSVR', 'Lasso', 'Lars', 'HuberRegressor', 'TheilSenRegressor']

In [ ]:
weights_markdown_str = """### Feature weights

Here are the feature weights as learned by the model."""

if model_name in skll_linear_models:
    display(Markdown(weights_markdown_str))

In [ ]:
if model_name in skll_linear_models:

    from rsmtool.modeler import Modeler
    from six import iteritems
    from sklearn.svm import SVR
    
    model_file = join(output_dir, '{}.model'.format(experiment_id))
    learner = Modeler.load_from_file(model_file).learner
    
    # get the coefficients and the intercept
    weights = {}
    coef = learner.model.coef_
    intercept = {'_intercept_': learner.model.intercept_}

    # convert SVR coefficient format (1 x matrix) to array
    if isinstance(learner._model, SVR):
        coef = coef.toarray()[0]

    # inverse transform to get indices for before feature selection
    coef = learner.feat_selector.inverse_transform(coef.reshape(1, -1))[0]
    for feat, idx in iteritems(learner.feat_vectorizer.vocabulary_):
        if coef[idx]:
            weights[feat] = coef[idx]

    # Some learners (e.g. LinearSVR) may return a list of intercepts
    if isinstance(intercept['_intercept_'], np.ndarray):
        intercept_list = ["%.12f" % i for i in intercept['_intercept_']]
        print("intercept = {}".format(intercept_list))
    else:
        print("intercept = {:.12f}".format(intercept['_intercept_']))
    print()
        
    print("Number of nonzero features:", len(weights))
    weight_items = iteritems(weights)
    for feat, val in sorted(weight_items, key=lambda x: -abs(x[1])):
        print("{:.12f}\t{}".format(val, feat))

## Evaluation results

### Overall association statistics

In [ ]:
markdown_str = ("The tables in this section show the standard association metrics between "
                "*observed* human scores and different types of machine scores. "
                "These results are computed on the evaluation set. `raw_trim` scores "
                "are truncated to [{}, {}]. `raw_trim_round` scores are computed by first truncating "
                "and then rounding the predicted score. Scaled scores are computed by re-scaling "
                "the predicted scores using mean and standard deviation of human scores as observed "
                "on the training data and mean and standard deviation of machine scores as predicted "
                "for the training set.".format(min_score, max_score))
display(Markdown(markdown_str))

#### Descriptive holistic score statistics

The table shows distributional properties of human and system scores. SMD values lower then -0.15 or higher than 0.15 are <span class="highlight_color">highlighted</span>.

*Please note that for raw scores, SMD values are likely to be affected by possible differences in scale.*

In [ ]:
raw_or_scaled = "scaled" if use_scaled_predictions else "raw"
eval_file = join(output_dir, '{}_eval.{}'.format(experiment_id, file_format))
df_eval = DataReader.read_from_file(eval_file, index_col=0)
distribution_columns = ['N', 'h_mean', 'sys_mean', 'h_sd',  'sys_sd', 'h_min', 'sys_min', 'h_max', 'sys_max', 'SMD']
association_columns = ['N'] + [column for column in df_eval.columns if not column in distribution_columns]
df_distribution = df_eval[distribution_columns]
df_association = df_eval[association_columns]

In [ ]:
pd.options.display.width=10
formatter = partial(color_highlighter, low=-0.15, high=0.15)
HTML('<span style="font-size:95%">'+ df_distribution.to_html(classes=['sortable'], 
                                                             escape=False,
                                                             formatters={'SMD': formatter},
                                                             float_format=float_format_func) + '</span>')

#### Association statistics


In [ ]:
markdown_str = ['The table shows the standard association metrics between human scores and machine scores.']
if continuous_human_score:
    markdown_str.append("Note that for computation of `kappa` both human and machine scores are rounded.")
else:
    markdown_str.append("Note that for computation of `kappa` all machine scores are rounded.")

Markdown('\n'.join(markdown_str))

In [ ]:
pd.options.display.width=10
HTML('<span style="font-size:95%">'+ df_association.to_html(classes=['sortable'], 
                                                            escape=False,
                                                            float_format=float_format_func) + '</span>')

### Confusion matrix

In [ ]:
markdown_str = ["Confusion matrix using {}, trimmed, and rounded scores and human scores (rows=human, columns=system).".format(raw_or_scaled)]

if continuous_human_score:
    markdown_str.append("Note: Human scores have beeen rounded to the nearest integer.")
            
Markdown('\n'.join(markdown_str))

In [ ]:
confmat_file = join(output_dir, '{}_confMatrix.{}'.format(experiment_id, file_format))
df_confmat = DataReader.read_from_file(confmat_file, index_col=0)
df_confmat

### Distribution of human and machine scores

In [ ]:
markdown_strs = ["The histogram and the table below show the distibution of "
                 "human scores and {}, trimmed, and rounded machine scores "
                 "(as % of all responses).".format(raw_or_scaled)]
markdown_strs.append("Differences in the table between human and machine distributions "
                     "larger than 5 percentage points are <span class='highlight_color'>highlighted</span>.")
if continuous_human_score:
    markdown_strs.append("Note: Human scores have beeen rounded to the nearest integer.")
    
display(Markdown('\n'.join(markdown_strs)))

In [ ]:
scoredist_file = join(output_dir, '{}_score_dist.{}'.format(experiment_id, file_format))
df_scoredist = DataReader.read_from_file(scoredist_file, index_col=0)
df_scoredist_melted = pd.melt(df_scoredist, id_vars=['score'])
df_scoredist_melted = df_scoredist_melted[df_scoredist_melted['variable'] != 'difference']

# get the colors for the plot
colors = sns.color_palette("Greys", 2)

with sns.axes_style('whitegrid'):

    # make a barplot without a legend since we will 
    # add one manually later
    p = sns.catplot(x="score", y="value", hue="variable", kind="bar",
                    palette=colors, data=df_scoredist_melted, 
                    height=3, aspect=2, legend=False)
    p.set_axis_labels('score', '% of responses')
    
    # add a legend with the right colors
    axis = p.axes[0][0]
    legend = axis.legend(labels=('Human', 'Machine'), title='', frameon=True, fancybox=True)
    legend.legend_handles[0].set_color(colors[0])
    legend.legend_handles[1].set_color(colors[1])

    imgfile = join(figure_dir, '{}_score_dist.svg'.format(experiment_id))
    plt.savefig(imgfile)

    if use_thumbnails:
        show_thumbnail(imgfile, next(id_generator))
    else:
        plt.show()

In [ ]:
formatter = partial(color_highlighter, low=0, high=5, absolute=True)
df_html = df_scoredist.to_html(classes=['sortable'], index=False, 
                               escape=False, formatters={'difference': formatter})
display(HTML(df_html))

In [ ]:
markdown_strs = ['### True score evaluations']

raw_or_scaled = "scaled" if use_scaled_predictions else "raw"
true_eval_file = join(output_dir, '{}_true_score_eval.{}'.format(experiment_id, file_format))
if exists(true_eval_file): 
    df_true_eval = DataReader.read_from_file(true_eval_file, index_col=0)
    df_true_eval.replace({np.nan: '-'}, inplace=True)
    prmse_columns = ['N','N raters', 'N single', 'N multiple', 
                     'Variance of errors', 'True score var',
                     'MSE true', 'PRMSE true']
    df_prmse = df_true_eval[prmse_columns]

    markdown_strs.append("The tables in this section show how well system scores can "
                         "predict *true* scores. According to Test theory, a *true* score "
                         "is a score that would have been obtained if there were no errors "
                         "in measurement. While true scores cannot be observed, the variance "
                         "of true scores and the prediction error can be estimated using observed "
                         "human scores when multiple human ratings are available for a subset of "
                         "responses.")

    if rater_error_variance is None: 
        
        rater_variance_source = 'estimated'
        # if we estimated rater error variance from the data,
        # we display the variance of the two human raters
        # so that the user can verify there is no bias
        # We get that data from existing analyses
        df_human_variance = df_consistency[['N', 'h1_sd', 'h2_sd']].copy()
        df_human_variance['N_double'] = df_human_variance['N']
        df_human_variance['h1_var (double)'] = df_human_variance['h1_sd']**2
        df_human_variance['h2_var (double)'] = df_human_variance['h2_sd']**2
        df_human_variance['N_total'] = df_eval.iloc[0]['N']
        df_human_variance['h1_var (single)'] = df_eval.iloc[0]['h_sd']**2
        df_human_variance.index = ['human']


        if context == 'rsmtool':
            label_column = "test_label_column"
        else:
            label_column = "human_score_column"

        markdown_strs.append("In this notebook the variance of true scores is estimated using "
                             "the human ratings available for "
                             "responses in the evaluation set. Note that the analyses in this "
                             "section assume that the values "
                            "in `{}` and `second_human_score_column` are independent scores "
                            "from different raters or groups of raters. These analyses are "
                            "not applicable to a situation where `{}` contains an average "
                            "score from multiple raters".format(label_column, label_column))

        markdown_strs.append("#### Variance of human scores")
        markdown_strs.append("The table below shows variance of both sets of human scores "
                            "for the whole evaluation set and for the subset of responses "
                            "that were double-scored. Large differences in variance between "
                            "the two human scores require further investigation.")
        display(Markdown('\n'.join(markdown_strs)))
        pd.options.display.width=10
        column_order = ['N_total', 'N_double', 'h1_var (single)', 'h1_var (double)', 'h2_var (double)']
        display(HTML('<span style="font-size:95%">'+ df_human_variance[column_order].to_html(classes=['sortable'], 
                                                                                            escape=False,
                                                                                            float_format=float_format_func) + '</span>'))
    else:
        markdown_strs.append("In this notebook the variance of true scores was "
                            "estimated using the value of rater error variance "
                            "supplied by the user ({})".format(rater_error_variance))
        display(Markdown('\n'.join(markdown_strs)))
        rater_variance_source = 'supplied'
    
    
    markdown_strs = ["#### Proportional reduction in mean squared error (PRMSE)"]
    markdown_strs.append("The table shows {} variance of human rater errors, "
                         "true score variance, mean squared error (MSE) and "
                         "proportional reduction in mean squared error (PRMSE) for "
                         "predicting a true score with system score. As for other evaluations, "
                         "these results are computed on the evaluation set. `raw_trim` scores "
                         "are truncated to [{}, {}]. `raw_trim_round` scores are computed "
                         "by first truncating and then rounding the predicted score. Scaled scores "
                         "are computed by re-scaling the predicted scores using mean and standard "
                         "deviation of human scores as observed on the training data and mean and "
                         "standard deviation of machine scores as predicted for the training set.".format(rater_variance_source,
                                                                                                          min_score,
                                                                                                          max_score))
    display(Markdown('\n'.join(markdown_strs)))
    pd.options.display.width=10
    display(HTML('<span style="font-size:95%">'+ df_prmse.to_html(classes=['sortable'], 
                                                               escape=False,
                                                               float_format=float_format_func) + '</span>'))
else:
    markdown_strs.append("The configuration file did not specify "
                         "`second_human_score_column` or `rater_error_variance`. "
                         "At least one of these must be specified to compute "
                         "evaluations against true scores.")
    display(Markdown('\n'.join(markdown_strs)))

## Principal component analysis

PCA using scaled data and singular value decomposition. This is computed using processed features after the truncation of outliers and other transformations specified in feature config file.

In [ ]:
pca_file = join(output_dir, '{}_pca.{}'.format(experiment_id, file_format))
df_pca = DataReader.read_from_file(pca_file, index_col=0)
df_pca.sort_index(inplace=True)
HTML(df_pca.to_html(classes=['sortable'], float_format=float_format_func))

In [ ]:
pcavar_file = join(output_dir, '{}_pcavar.{}'.format(experiment_id, file_format))
df_pcavar = DataReader.read_from_file(pcavar_file, index_col=0)
df_pcavar.sort_index(inplace=True)
HTML(df_pcavar.to_html(classes=['sortable'], float_format=float_format_func))

In [ ]:
# generate the Scree plot
with sns.axes_style('white'):
    num_components = len(df_pcavar.columns)
    labels = list(df_pcavar.columns)
    ax = df_pcavar.transpose().plot(y='Eigenvalues', kind='line', 
                                    color='black', linestyle='dashed', marker='o', legend=False,
                                    linewidth=1, use_index=True, xticks=range(num_components),
                                    figsize=(11, 5), title='Scree Plot: Principal Component Analysis')
    ax.set_ylabel('Variances')
    ax.set_xticks(ax.get_xticks(), labels, rotation=90)
    imgfile = join(figure_dir, '{}_pca.svg'.format(experiment_id))
    plt.savefig(imgfile)
    if use_thumbnails:
        show_thumbnail(imgfile, next(id_generator))
    else:
        plt.show()

## Links to intermediate files

In [ ]:
# dynamically set the documentation link based on context
if context == "rsmtool":
    documentation_link = "https://rsmtool.readthedocs.io/en/main/usage_rsmtool.html#intermediate-files"
else:
    # this is rsmeval
    documentation_link = "https://rsmtool.readthedocs.io/en/main/advanced_usage.html#intermediate-files"

markdown_text = ("The following intermediate files were generated as part of this experiment."
"To examine a file, click on the link. Note that the descriptions provided here are only brief "
"summaries. For more detailed descriptions, please refer to the "
f"[relevant RSMTool documentation]({documentation_link}).")

In [ ]:
display(Markdown(markdown_text))

In [ ]:
from rsmtool.utils.notebook import show_files

In [ ]:
show_files(context, output_dir, experiment_id, file_format)

## System information

In [ ]:
system_name = platform.system()

# People might not know what 'Darwin' is, so we should replace that with 'Mac OS X'
if system_name == 'Darwin':
    system_name = 'Mac OS X'
    
# get the architecture
architecture = platform.architecture()[0]

# get the rsmtool version
rsmtool_version_str = '.'.join(map(str, rsmtool_version))

display(Markdown('This report was generated using rsmtool v{} on a '
                 '{} computer running {}.'.format(rsmtool_version_str, 
                                                  architecture, 
                                                  system_name)))

### Python packages

In [ ]:
from importlib.metadata import distributions
package_names = '\n'.join(sorted(["%s==%s" % (d.metadata['name'], d.version) for d in distributions()]))
display(HTML('<div id="packages"><pre>{}</pre></div>'.format(package_names)))

In [ ]:
%%javascript

// Code to dynamically generate table of contents at the top of the HTML file
var tocEntries = ['<ul>'];
var anchors = $('a.anchor-link');
var headingTypes = $(anchors).parent().map(function() { return $(this).prop('tagName')});
var headingTexts = $(anchors).parent().map(function() { return $(this).text()});
var subList = false;

$.each(anchors, function(i, anch) {
    var hType = headingTypes[i];
    var hText = headingTexts[i];
    hText = hText.substr(0, hText.length - 1);
    if (hType == 'H2') {
        if (subList) {
            tocEntries.push('</ul>')
            subList = false;
        }
        tocEntries.push('<li><a href="' + anch + '"</a>' + hText + '</li>')
    }
    else if (hType == 'H3') {
        if (!subList) {
            subList = true;
            tocEntries.push('<ul>')
        }
        tocEntries.push('<li><a href="' + anch + '"</a>' + hText + '</li>')
    }
});
tocEntries.push('</ul>')
$('#toc').html(tocEntries.join(' '))